<a href="https://colab.research.google.com/github/karl-karlsson/notebooks/blob/main/laptops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can open a copy of this notebook in Google Colab by clicking the icon above

-----

**This project is ongoing, ie. not finished**

![laptop](https://freesvg.org/img/1520464168.png)

# Building fast queries on a CSV

This was originally a guided project from Dataquest ([link](https://www.dataquest.io/)). 

I've downloaded a dataset on laptops to my GitHub respository from Kaggle ([info about dataset](https://www.kaggle.com/datasets/ionaskel/laptop-prices)).

I wanted to do the project in Google Colab (rather than on the Dataquest platform) so I've had to make some adjustments with regards to loading in the data. Other than that, however, this is very much a *guided* project. So I don't want to take credit for solutions etc. 

I've done the project for my own learning experience.

# Scenario
We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory. The `laptops.csv` file is our inventory. Let's start by:

* Downloading the file from GitHub using the `requests` package
* Read in the file using the `csv` package 
* Print out the first five rows



In [101]:
import csv
import requests

url = 'https://raw.githubusercontent.com/karl-karlsson/data/main/laptops.csv'


with requests.Session() as s:
    download = s.get(url)

    decoded_content = download.content.decode('latin-1')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

for row in range(5):
  print(my_list[row])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']
['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69']
['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94']
['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575.00']
['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45']
